In [ ]:
import numpy as np
import mcmc
from scipy import stats
import invchi2

In [ ]:
nu = 4
n = 10

model = lambda mu, sigma: stats.norm.rvs(mu, invchi2.rvs(nu, sigma))
    
y = [model(13, 2) for _ in range(n)]

In [ ]:
def start():
    mu = stats.uniform.rvs(0, 100)
    sigma = stats.uniform.rvs(0, 100)
    a = [mu, sigma]
    for _ in range(n):
        a.append(invchi2.rvs(nu, sigma))
    return a

In [ ]:
iterations = 8000 # must be divisible by 4

def update(theta):
    V = np.array(theta[2:])
    precision = np.sum(1.0 / V)

    # udpdate mu
    loc = np.sum(y / V) / precision
    scale = np.sqrt(1 / precision)
    theta[0] = stats.norm.rvs(loc, scale)

    # udpdate sigma
    beta = nu * precision / 2.0
    theta[1] = np.sqrt(stats.gamma.rvs(n * nu / 2.0, scale = 1 / beta))

    # udpdate V
    mu = theta[0]
    sigma = theta[1]
    V_new = invchi2.rvs(nu + 1, np.sqrt((nu * sigma ** 2 + (y - mu) ** 2) / (nu + 1)))
    for i, v in enumerate(V_new):
        theta[2 + i] = v

starts = [start() for _ in range(5)]
samples, rhat, acceptance = mcmc.run(starts, iterations, update)
print rhat
samples.shape